In [3]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain langchain_community langchain bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import ChatOpenAI

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

In [6]:
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = Ollama(model='llama3')

In [7]:

loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/get_started/introduction")


In [8]:
docs = loader.load()

docs

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentLangChain v0.2 is out! You are currently viewing the old v0.1 docs. View the latest docs here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionQuickstartInstallationUse casesQ&A with RAGExtracting structured outputChatbotsTool use and agentsQuery analysisQ&A over SQL + CSVMoreExpression LanguageGet startedRunnable interfacePrimitivesAdvantages of LCELStreamingAdd message history (memory)MoreEcosystem🦜🛠️ LangSmith🦜🕸️ LangGraph🦜️🏓 LangServeSecurit

In [9]:
# embeddings = OpenAIEmbeddings()
embeddings = OllamaEmbeddings()

In [11]:
%pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from langchain_pinecone import PineconeVectorStore

index_name = "test-index"
namespace = "my-namespace"
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings,
    pinecone_api_key="b0fc8f7a-963f-402d-9722-035d663d39db"
 
)


# index_name = "test-index"
# api_key = ''
# docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name, api_key="b0fc8f7a-963f-402d-9722-035d663d39db")

In [17]:
query = "What is langchain"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025B93D964D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
retriever = docsearch.as_retriever(search_type="mmr")

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")